In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
from helper_functions import *
import sys
import os

folder = "./data_files"
files = [f for f in os.listdir(folder) if f.endswith(".bed")][1:]

# files.sort()
# files = files[:14]
# files

# f
# HG001 - Female. Individual

# NA19240 - Female. Individual

# HG002 - Male. Son of HG003 and HG004
# HG003 - Male. Father of HG002
# HG004 - Female. Mother of HG002

# HG005 - Male. Son of HG006 and HG007
# HG006 - Male. Father of HG005
# HG007 - Female. Mother of HG005


In [ ]:

trio1 = dict(father=['HG003_copynumber_variants.bed','HG003_short_variants.bed'][1],
            child=['HG002_copynumber_variants.bed', 'HG002_short_variants.bed'][1],
            mother=['HG004_copynumber_variants.bed', 'HG004_short_variants.bed'][1])

t1 = dict()
for k,v in trio1.items():
    t1[k] = []
    print(k, v)
    _f = read_tsv(os.path.join(folder, v))
    _f["hash"] = _f.iloc[:,[1,2,4]].apply(lambda x: hash(tuple(x)), axis=1)
    _f["chr_hash"] = _f.iloc[:,[0,1,2,3,4]].apply(lambda x: hash(tuple(x)), axis=1)
    t1[k] = _f
        
exons = pd.read_csv("hg19_exon_locations_with_genes.csv")

mother_father_hashes = t1["mother"][t1["mother"]["hash"].isin(t1["father"]["hash"])]["hash"]
interesting_short = t1["son"][~t1["son"]["hash"].isin(mother_father_hashes)]


interIter = interesting_short.iterrows();
inter = next(interIter)

# differences in either parent or just child
interIndices = []

for index, row in exons.iterrows():
    while(inter[1].START_POSITION <= row.END_POSITION):
        if (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION) \
        | \
        (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION):
             interIndices.append(inter[0])
        inter = next(interIter)

In [5]:
%%time

from tqdm.autonotebook import tqdm

def make_bed_df(fullpath):
    print("Reading:", fullpath)
    _f = read_tsv(fullpath)
#     tqdm.pandas(desc="Hash")
#     _f["hash"] = _f.iloc[:,[1,2,4]].progress_apply(lambda x: hash(tuple(x)), axis=1)
    _f["hash"] = _f.iloc[:, 1].astype(str) + _f.iloc[:, 2].astype(str) + _f.iloc[:, 4].astype(str)
#     tqdm.pandas(desc="Chr Hash")
    _f["chr_hash"] = _f.iloc[:, 0].astype(str) + _f.iloc[:, 1].astype(str) + _f.iloc[:, 2].astype(str) + _f.iloc[:, 3].astype(str) + _f.iloc[:, 4].astype(str)
#     _f["chr_hash"] = _f.iloc[:,[0,1,2,3,4]].progress_apply(lambda x: hash(tuple(x)), axis=1)
    return _f

def make_trio_dfs(father: str, mother: str, child: str):
    father = make_bed_df(father)
    mother = make_bed_df(mother)
    child = make_bed_df(child)
    return father, mother, child

def adam_func(father: pd.DataFrame, mother: pd.DataFrame, child: pd.DataFrame):
    exons = pd.read_csv("hg19_exon_locations_with_genes.csv")
    
#     set_mother = set(mother["hash"])
#     set_father = set(father["hash"])
#     set_child = set(child["hash"])
#     intercet = set.intersection(set_mother, set_father, set_child)
#     interesting_short = child[child["hash"].isin(intercet)]
    mother_father_hashes = mother[mother["hash"].isin(father["hash"])]["hash"]
    interesting_short = child[~child["hash"].isin(mother_father_hashes)]
#     mother_father_hashes = mother[mother["hash"].isin(father["hash"])]["hash"]
#     interesting_short = son[~son["hash"].isin(mother_father_hashes)]


    interIter = interesting_short.iterrows();
    inter = next(interIter)

    # differences in either parent or just child
    interIndices = []

    for index, row in tqdm(exons.iterrows()):
        while(inter[1].START_POSITION <= row.END_POSITION):
            if (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION) \
            | \
            (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION):
                 interIndices.append(inter[0])
            inter = next(interIter)
    return child.iloc[interIndices]

def vicky_func(father: pd.DataFrame, mother: pd.DataFrame, child: pd.DataFrame, lb=0.7, ub=1.04):
    child = child[(child.iloc[:, 5]==0) & (child.iloc[:, 6] > 0)]
    set_child = set(child["chr_hash"])
    set_mother = set(mother["chr_hash"])
    set_father = set(father["chr_hash"])

    family = set.intersection(set_child, set_mother, set_father)
    father = father[father["chr_hash"].isin(family)]
    mother = mother[mother["chr_hash"].isin(family)]
    child = child[child["chr_hash"].isin(family)]

    mother["vratio"] = mother.iloc[:,5] / mother.iloc[:, 6]
    father["vratio"] = father.iloc[:,5] / father.iloc[:, 6]
    return  father[(father["vratio"] > .7) & (father["vratio"] < 1.3)], mother[(mother["vratio"] > lb) & (mother["vratio"] < ub)], father[(father["vratio"] > .7) & (father["vratio"] < 1.3)]

def all_together(father: str, mother: str, child: str, save_path: str):
    
    father, mother, child = make_trio_dfs(**trio1)

    a_child = adam_func(father, mother, child)

    v_father, v_mother, v_child = vicky_func(father, mother, child)
    
    vfather = os.path.join(save_path, "v_father.bed") 
    vmother = os.path.join(save_path, "v_mother.bed") 
    vchild = os.path.join(save_path, "v_child.bed") 
    achild = os.path.join(save_path, "a_child.bed") 
    v_father.iloc[:, :6].to_csv(vfather, index=False, sep="\t")
    v_mother.iloc[:, :6].to_csv(vmother, index=False, sep="\t")
    v_child.iloc[:, :6].to_csv(vchild, index=False, sep="\t")
    
    a_child.iloc[:, :6].to_csv(achild, index=False)
    return vfather, vmother, vchild, achild
    

trio1 = dict(father=['HG003_copynumber_variants.bed',os.path.join(folder,'HG003_short_variants.bed')][1],
            child=['HG002_copynumber_variants.bed', os.path.join(folder,'HG002_short_variants.bed')][1],
            mother=['HG004_copynumber_variants.bed', os.path.join(folder,'HG004_short_variants.bed')][1])


out = all_together(save_path=os.getcwd(), **trio1)

Reading: ./data_files/HG003_short_variants.bed
Reading: ./data_files/HG004_short_variants.bed
Reading: ./data_files/HG002_short_variants.bed


/home/mcamp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mcamp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 1min 33s, sys: 4.04 s, total: 1min 37s
Wall time: 1min 37s


In [4]:
out

('/home/mcamp/code-home/Make-It-Pop-HATCH-2020/v_father.bed',
 '/home/mcamp/code-home/Make-It-Pop-HATCH-2020/v_mother.bed',
 '/home/mcamp/code-home/Make-It-Pop-HATCH-2020/v_child.bed',
 '/home/mcamp/code-home/Make-It-Pop-HATCH-2020/a_child.bed')

In [3]:
%%time
trio1 = dict(father=['HG003_copynumber_variants.bed','HG003_short_variants.bed'][1],
            son=['HG002_copynumber_variants.bed', 'HG002_short_variants.bed'][1],
            mother=['HG004_copynumber_variants.bed', 'HG004_short_variants.bed'][1])

# t1 = dict()
# for k,v in trio1.items():
#     t1[k] = []
#     print(k, v)
#     _f = read_tsv(os.path.join(folder, v))
#     _f["hash"] = _f.iloc[:,[1,2,4]].apply(lambda x: hash(tuple(x)), axis=1)
#     _f["chr_hash"] = _f.iloc[:,[0,1,2,3,4]].apply(lambda x: hash(tuple(x)), axis=1)
#     t1[k] = _f
        
exons = pd.read_csv("hg19_exon_locations_with_genes.csv")

mother_father_hashes = t1["mother"][t1["mother"]["hash"].isin(t1["father"]["hash"])]["hash"]
interesting_short = t1["son"][~t1["son"]["hash"].isin(mother_father_hashes)]


interIter = interesting_short.iterrows();
inter = next(interIter)

# differences in either parent or just child
interIndices = []

for index, row in exons.iterrows():
    while(inter[1].START_POSITION <= row.END_POSITION):
        if (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION) \
        | \
        (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION):
             interIndices.append(inter[0])
        inter = next(interIter)

father HG003_short_variants.bed
son HG002_short_variants.bed
mother HG004_short_variants.bed
CPU times: user 5min 47s, sys: 2.86 s, total: 5min 50s
Wall time: 5min 51s


In [5]:
child = t1["son"]
child = child[(child.iloc[:, 5]==0) & (child.iloc[:, 6] > 0)]
set_child = set(child["chr_hash"])
set_mother = set(mother["chr_hash"])
set_father = set(father["chr_hash"])

family = set.intersection(set_child, set_mother, set_father)
father = father[father["chr_hash"].isin(family)]
mother = mother[mother["chr_hash"].isin(family)]
child = child[child["chr_hash"].isin(family)]


(201390, 9)

In [53]:
father = t1["father"]
mother = t1["mother"]
set_child = set(child["chr_hash"])
set_mother = set(mother["chr_hash"])
set_father = set(father["chr_hash"])

family = set.intersection(set_child, set_mother, set_father)
father = father[father["chr_hash"].isin(family)]
mother = mother[mother["chr_hash"].isin(family)]
child = child[child["chr_hash"].isin(family)]

mother.shape, father.shape, child.shape

((163879, 9), (163879, 10), (163879, 9))

In [42]:
father[father["chr_hash"].isin(child["chr_hash"])]
mother = mother[mother["chr_hash"].isin(child["chr_hash"])]
print(mother.shape[0], father.shape[0])
father = father[father["chr_hash"].isin(father["chr_hash"])]
mother = mother[mother["chr_hash"].isin(mother["chr_hash"])]
mother.shape[0], father.shape[0]

179099 3600577


(179099, 3600577)

In [51]:
def vratio(x):
    if x[6] > 0:
        return x[5] / x[6]
    else:
        return -1

mother["vratio"] = mother.iloc[:,5] / mother.iloc[:, 6]
father["vratio"] = father.iloc[:,5] / father.iloc[:, 6]
mother[(mother["vratio"] > .7) & (mother["vratio"] < 1.3)].shape, father[(father["vratio"] > .7) & (father["vratio"] < 1.3)].shape
# father["vratio"] = father.apply(lambda x: vratio(x), axis=1)

((37901, 10), (38217, 10))

In [39]:
father["vratio"].describe()

count    3.562136e+06
mean              inf
std               NaN
min      0.000000e+00
25%      1.823899e-01
50%      9.132075e-01
75%      1.041026e+00
max               inf
Name: vratio, dtype: float64

In [52]:
father[(father["vratio"] > .9) & (father["vratio"] < 1.04)].groupby("CHROMOSOME_NAME").count()

,START_POSITION,END_POSITION,REFERENCE,ALTERNATE,REFERECE_READS,ALTERNATE_READS,hash,chr_hash,vratio
CHROMOSOME_NAME,,,,,,,,,
chr1,1155,1155,1155,1155,1155,1155,1155,1155,1155
chr10,642,642,642,642,642,642,642,642,642
chr11,831,831,831,831,831,831,831,831,831
chr12,818,818,818,818,818,818,818,818,818
chr13,470,470,470,470,470,470,470,470,470
chr14,480,480,480,480,480,480,480,480,480
chr15,423,423,423,423,423,423,423,423,423
chr16,380,380,380,380,380,380,380,380,380
chr17,364,364,364,364,364,364,364,364,364


In [116]:
interIter = interesting_short.iterrows();
inter = next(interIter)
interIndices = []

for index, row in exons.iterrows():
    while(inter[1].START_POSITION <= row.END_POSITION):
        if (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION) \
        | \
        (inter[1].START_POSITION >= row.START_POSITION & inter[1].START_POSITION < row.END_POSITION):
             interIndices.append(inter[0])
        inter = next(interIter)

In [98]:
t1["father"][0].head()

,CHROMOSOME_NAME,START_POSITION,END_POSITION,REFERENCE,ALTERNATE,REFERECE_READS,ALTERNATE_READS,hash
0,chr1,54711,54712,2,3,NaN,NaN,-6758647717633582781
1,chr1,532076,532177,2,1,NaN,NaN,-6623031206355565155
2,chr1,724860,724861,2,3,NaN,NaN,3806466207174313671
3,chr1,756258,756259,2,3,NaN,NaN,-2534817748319624213
4,chr1,757806,757842,2,1,NaN,NaN,-8254567945178617948


In [ ]:
import pandas;
import pandasql as ps;
import numpy as np;
pandas.__version__
def load_short_tsv(filename):
    dataPath  = "./make_it_pop/data_files/"
    filePath = dataPath+filename
    tsvPandas = pandas.read_csv(filePath,'\t',header=None)
    columnNames = [f"col{i}" for i in range(len(tsvPandas.columns))]
    tsvPandas.columns = columnNames
    return tsvPandas
exonsWithGenes = pandas.read_csv("./make_it_pop/data_files/hg19_exon_locations_with_genes.csv")
print(exonsWithGenes)
sonShort = load_short_tsv("HG002_short_variants.tsv")
print(sonShort)
fatherShort = load_short_tsv("HG003_short_variants.tsv")
print(fatherShort)
motherShort = load_short_tsv("HG004_short_variants.tsv")
print(motherShort)
motherShort["hash"] = motherShort["col1"].astype(str) + motherShort["col2"].astype(str) + motherShort["col4"].astype(str)
fatherShort["hash"] = fatherShort["col1"].astype(str) + fatherShort["col2"].astype(str) + fatherShort["col4"].astype(str)
sonShort["hash"] =  sonShort["col1"].astype(str) + sonShort["col2"].astype(str) + sonShort["col4"].astype(str)
motherAndFatherHashes = motherShort[motherShort["hash"].isin(fatherShort["hash"])]["hash"]
interestingShort = sonShort[~sonShort["hash"].isin(
    motherAndFatherHashes
)]
print(interestingShort)

interIter = interestingShort.iterrows();
inter = next(interIter)
interIndices = []
# print(interestingShort.loc[interestingIndex])
for index, row in exonsWithGenes.iterrows():
    while(inter[1].col1 <= row.END_POSITION):
        if (inter[1].col1 >= row.START_POSITION & inter[1].col1 < row.END_POSITION) | (inter[1].col2 >= row.START_POSITION & inter[1].col2 < row.END_POSITION):
             interIndices.append(inter[0])
        inter = next(interIter)
print(len(interIndices))

In [55]:
hg02 = read_tsv("./data_files/HG002_copynumber_variants.tsv")
hg02[hg02["CHROMOSOME_NAME"] == "chr11"]

FileNotFoundError: [Errno 2] File b'./data_files/HG002_copynumber_variants.tsv' does not exist: b'./data_files/HG002_copynumber_variants.tsv'

In [48]:
import numpy as np

np.array_split(h19["CHROMOSOME_NAME"].values, int(round(h19.shape[0]/11)))

[array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1',
        'chr1', 'chr1', 'chr1'], dtype=object),
 array(['chr1', 

In [34]:
hg02_short = read_tsv("./data_files/HG002_short_variants.tsv")
hg02_short[hg02_short["CHROMOSOME_NAME"] == "chr11"]

,CHROMOSOME_NAME,START_POSITION,END_POSITION,REFERENCE,ALTERNATE,REFERECE_READS,ALTERNATE_READS
2355316,chr11,201026,201027,C,A,77,420
2355317,chr11,204061,204062,T,C,100,169
2355318,chr11,204099,204100,T,C,44,29
2355319,chr11,204146,204147,G,A,0,65
2355320,chr11,204967,204968,G,A,116,359
2355321,chr11,204985,204986,C,A,128,366
2355322,chr11,205197,205198,C,A,0,322
2355323,chr11,207358,207359,C,CGGCCACAGCCGCCTCAGACGT,56,56
2355324,chr11,207399,207400,A,G,65,65
2355325,chr11,207461,207462,A,G,74,387


In [5]:
h19 = read_tsv("./data_files/hg19_genes.tsv")
h19["ix"] = h19[["START_POSITION" , "END_POSITION"]].apply(tuple, axis=1)

In [6]:
h19a[h19["START_POSITION"] >= ix[0]]

NameError: name 'h19a' is not defined

In [7]:
exon_ix["gene"] = h19[h19["ix"] == ix]["REFERENCE"].iloc[0]
exon_ix

NameError: name 'ix' is not defined

,CHROMOSOME_NAME,START_POSITION,END_POSITION,REFERENCE,ALTERNATE,REFERECE_READS,ALTERNATE_READS,ix
0,chr1,11873,14409,DDX11L1,NaN,NaN,NaN,"(11873, 14409)"
1,chr1,14361,29370,WASH7P,NaN,NaN,NaN,"(14361, 29370)"
2,chr1,17368,17436,MIR6859-1,NaN,NaN,NaN,"(17368, 17436)"
3,chr1,17368,17436,MIR6859-2,NaN,NaN,NaN,"(17368, 17436)"
4,chr1,17368,17436,MIR6859-3,NaN,NaN,NaN,"(17368, 17436)"
5,chr1,17368,17436,MIR6859-4,NaN,NaN,NaN,"(17368, 17436)"
6,chr1,30365,30503,MIR1302-10,NaN,NaN,NaN,"(30365, 30503)"
7,chr1,30365,30503,MIR1302-2,NaN,NaN,NaN,"(30365, 30503)"
8,chr1,30365,30503,MIR1302-11,NaN,NaN,NaN,"(30365, 30503)"
9,chr1,30365,30503,MIR1302-9,NaN,NaN,NaN,"(30365, 30503)"


In [29]:
import numpy as np
import pandas as pd

df = pd.read_csv("/home/mcamp/code-home/Make-It-Pop-HATCH-2020/test_condition_gene.txt", delimiter="\t")
df = df[df["object"] == "condition"]
df = df[df["object_name"] != "Hereditary disear"]
# df.groupby("gene_symbol").count()
df["gene_symbol"].fillna("wtf!").value_counts()

wtf!    231153
Name: gene_symbol, dtype: int64

In [28]:
df["object_name"].value_counts()

Hereditary disease                                                                        15076
Malignant tumor of prostate                                                                 434
Cystic fibrosis                                                                             406
Familial cancer of breast                                                                   343
Ehlers-Danlos syndrome, classic type                                                        342
Acute myeloid leukemia                                                                      286
Angelman syndrome                                                                           283
Breast-ovarian cancer, familial 2                                                           270
Leigh syndrome                                                                              269
Carcinoma of colon                                                                          265
Breast-ovarian cancer, familial 1       

In [15]:
ref = pd.read_csv("/home/mcamp/code-home/Make-It-Pop-HATCH-2020/hg19_exon_locations_with_genes.csv")
ref["genes"].value_counts()

DMD             5399
CSMD1           2998
IL1RAPL1        2215
RFX3            1892
PTPRD           1692
DHRSX           1382
GLIS3           1205
IL1RAPL2        1173
SNTG1           1127
PCDH11Y         1087
PTCHD1-AS       1059
MID1             978
PDE4D            969
PXDNL            957
LINGO2           905
SPIDR            896
DIAPH2           881
LINC00278        865
XKR4             831
ERICH1-AS1       807
HECW1            792
EFHC2            785
EYS              782
SUGCT            765
NHS              758
ARHGAP6          752
DACH2            731
ARHGEF10         727
CASK             712
FRMPD4           703
                ... 
LIN54              1
IRS1               1
SRGAP2C            1
OSGEPL1-AS1        1
CCDC166            1
BLACAT1            1
TLR5               1
LINC01591          1
LINC01031          1
COG2               1
XXYLT1-AS2         1
KIT                1
GCSH               1
LOC401463          1
MTRNR2L10          1
LINC01324          1
CAMK2N2      

# Sample desired output data (for now)
Output is mainly modification of patient data (human subjects HG003 and HG006)
Below sample is based on HG002_shortvariants
Column 8 would classify variant's location as "exon", "intron", or "intergenic"
Column 9 would list the corresponding gene name, if applicable (NA if not applicable)

```
chr1	118616	118617	T	C	0	0	Intergenic	NA
```

In [8]:
from tqdm.autonotebook import tqdm
h19a = read_tsv("./data_files/hg19_exon_locations.tsv")


for ix in tqdm(h19["ix"].drop_duplicates().values):    
    exon_ix = h19a[(h19a["START_POSITION"] >= ix[0]) & (h19a["END_POSITION"] < ix[1])]
    h19a.at[exon_ix.index, "genes"] = h19[h19["ix"] == ix]["REFERENCE"].iloc[0]
    



/home/mcamp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


In [ ]:

To get desired output:
Compare hg19 exon locations to hg19 gene locations. Add gene names to hg19 exon file to create a modified hg19_exon_locations file.
Compare patient's short variant file to modified hg19 exon locations file
Create output as a modified version of the short_variant file, with new columns 8 and 9
Sort the modified short_variants such that:
Exon variants appear first, then introns, then intergenic sequences
Sort within each category by gene start position
FIGURE OUT IF/HOW WE WANT TO CLASSIFY HOMOZYGOUS AND HETEROZYGOUS
Figure out read ratio bounds to classify patient variants as homozygous or heterozygous
Process patient short variant file
Process parent short variant files for homozygous and heterozygous sequences
Questions to ask moderators/challenge writer
-What's with the 0	0 lines in short variant files?
-How shold we deal with the ratios between reference and alternate reads?